In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.nn as nn
import json
import pickle
%cd /content/drive/MyDrive/인공지능/NLP

/content/drive/MyDrive/인공지능/NLP


In [4]:
with open("./MovieReview_SentimentAnalysis-main/top_10000_index.json", encoding="utf-8") as F:
  topN_index = json.load(F)
  look_up_table = topN_index

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class LSTM(nn.Module):
    def __init__(self,embeded_size):
        super(LSTM,self).__init__()
        self.embeded_size = embeded_size
        self.embedding_layer = nn.Embedding(self.embeded_size,
                                           embedding_dim = 100)
        self.hidden_size = 80
        self.lstm = nn.LSTM(100,self.hidden_size,bidirectional=True,batch_first=True)
        self.linear = nn.Linear(self.hidden_size*2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        embedding = self.embedding_layer(x)
        lstm_out, (ht,ct) = self.lstm(embedding)
        
        out = self.linear(lstm_out[:,-1,:])
        return self.sigmoid(out)
        

In [7]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [8]:
net = LSTM(len(look_up_table)).to(device)

In [9]:
import numpy as np
train_x = np.load("./train_x_save.npy")
train_y = np.load("./train_y_save.npy")
test_x = np.load("./test_x_save.npy")
test_y = np.load("./test_y_save.npy")

In [10]:
tx = torch.LongTensor(train_x)
ty = torch.FloatTensor(train_y)

dx = torch.LongTensor(test_x)
dy = torch.FloatTensor(test_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(145791, 78)
(145791,)
(48995, 76)


In [11]:
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
batch_size = 500

train_data_set = TensorDataset(tx,ty)
test_data_set = TensorDataset(dx,dy)

train_data_loader = torch.utils.data.DataLoader(train_data_set,batch_size,shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_data_set,batch_size,shuffle=True)

epoch = 15
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
lossF = torch.nn.BCELoss().to(device)

for i in tqdm(range(0,epoch)):
  for (train_x,train_y) in train_data_loader:
    train_x,train_y = train_x.to(device),train_y.to(device)
    y_pred = net(train_x)
    loss = lossF(y_pred,train_y.view(-1,1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  accuracy = 0
  for (test_x,test_y) in test_data_loader:
    test_x = test_x.to(device)
    y_pred = net(test_x)
    pred = np.zeros_like(y_pred.detach().cpu().numpy())
    mask = (y_pred.detach().cpu().numpy() >= 0.5)
    pred[mask] = 1
    pred = torch.FloatTensor(pred)
    accuracy += (pred.view(-1) == test_y).to(float).sum()

  print(f'loss : {loss.mean()} accuracy {100*accuracy.item()/48995}:')

loss : 0.39595362544059753 accuracy 81.61445045412798:
loss : 0.34688735008239746 accuracy 83.66976222063475:
loss : 0.35456523299217224 accuracy 84.4024900500051:
loss : 0.35108986496925354 accuracy 84.86172058373303:
loss : 0.24609312415122986 accuracy 85.14134095315849:
loss : 0.22781576216220856 accuracy 84.92295132156343:
loss : 0.20050086081027985 accuracy 85.05765894479029:
loss : 0.2166140377521515 accuracy 84.70252066537402:
loss : 0.3022626042366028 accuracy 84.73925910807225:
loss : 0.2434493899345398 accuracy 84.59842841106236:
loss : 0.15379421412944794 accuracy 84.21063373813655:
loss : 0.16819116473197937 accuracy 84.06980304112665:
loss : 0.11259102821350098 accuracy 84.17185427084397:
loss : 0.09189334511756897 accuracy 83.8697826308807:
loss : 0.11547905206680298 accuracy 83.77997754872946:

